### Cleaning the Inspection Data:
The first part of this is cleaning the inspection data and creating a new csv file

In [ ]:
# Import dependencies
from pprint import pprint
import pandas as pd 
import requests
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
from sklearn import datasets

#yelp_api_key = 

In [ ]:
#import food inspection CSV file
inspection = pd.read_csv('inspection_5years.csv')
inspection.head(5)

In [ ]:
#Change the format of the address
inspection = pd.DataFrame(inspection, columns = ['Address','AKA Name','Facility Type','Risk',
                                                'Zip', 'Year','Inspection Type','Results','Violations','Latitude','Longitude'])
inspection['Address'] = inspection['Address'].str.lower()

print(inspection)

In [ ]:
inspection = pd.DataFrame(inspection, columns = ['Address','AKA Name','Facility Type','Risk',
                                                'Zip', 'Year','Inspection Type','Results','Violations','Latitude','Longitude'])

inspection['Address'] = inspection['Address'].str.title()

print(inspection)

In [ ]:
inspection.to_csv('inspection_clean.csv', index=False)

In [ ]:
#list all the zipcodes
zipcode = inspection['Zip'].value_counts()
zipcode